In [2]:
# noexport

!typech evaluation_utils.ipynb


[NbConvertApp] Converting notebook evaluation_utils.ipynb to python


In [17]:
from train_utils import *

In [16]:
from getsecret import getsecret
import subprocess
import json

In [27]:
#print(get_models_currently_training())

In [15]:
def get_models_currently_training():
  output = []
  local_info = json.load(open('current.json'))
  output.append(local_info['base_path'])
  remote_info = json.loads(subprocess.check_output(getsecret('REMOTE_SSH_COMMAND'), shell=True))
  output.append(remote_info['base_path'])
  return output

In [9]:
from glob import glob
import os
from os import path

In [11]:
def find_last_completed_epoch(model_name):
  return len(glob(path.join(model_name, 'info_*.json')))


In [30]:
#print(get_models_with_training_status())


In [22]:


def get_models_with_training_status():
  output = []
  models_currently_training = set(get_models_currently_training())
  for x in glob('tm_*'):
    model_status = {
      'name': x,
      'epochs_done': find_last_completed_epoch(x),
      'status': 'incomplete'
    }
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      output.append(model_status)
      continue
    status_info = json.load(open(status_file_path))
    if status_info['status'] == 'done':
      #model_status['epochs_done'] = find_last_completed_epoch(x)
      model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    if status_info['status'] == 'training':
      if x in models_currently_training:
        model_status['status'] = status_info['status']
      output.append(model_status)
      continue
    raise Exception('invalid status for model', x, status_info['status'])
    #output.append(model_status)
  return output

In [28]:
import shutil

def remove_incomplete_models():
  for status_info in get_models_with_training_status():
    if status_info['epochs_done'] == 0 and status_info['status'] == 'incomplete':
      model_name = status_info['name']
      shutil.rmtree(model_name)

remove_incomplete_models()

In [40]:
def get_parameter_info_list_for_model(model_name):
  parameters_file_path = path.join(model_name, 'parameters.json')
  return json.load(open(parameters_file_path))

def get_parameter_map_for_model(model_name):
  output = {}
  for parameter_info in get_parameter_info_list_for_model(model_name):
    parameter_name = parameter_info['name']
    parameter_value = parameter_info['value']
    output[parameter_name] = parameter_value
  return output

def list_models_finished_training():
  output = []
  for x in glob('tm_*'):
    status_file_path = path.join(x, 'status.json')
    if not os.path.exists(status_file_path):
      continue
    status_info = json.load(open(status_file_path))
    #print(status_info)
    if status_info['status'] != 'done':
      continue
    output.append(x)
  return output

def list_models_matching_parameters(parameters_map):
  output = []
  for model_name in list_models_finished_training():
    parameters_map_cur = get_parameter_map_for_model(model_name)
    is_match = True
    for k,v in parameters_map.items():
      if parameters_map_cur.get(k, None) != v:
        is_match = False
        break
    if is_match:
      output.append(model_name)
  return output


def get_models_with_dev_correct_fraction():
  #for x in list_models_finished_training():
  #  get_parameter_map_for_model(x))
  output = []
  for model_name in list_models_matching_parameters({'enable_current_difficulty': False}):
    for info_file_name in glob(path.join(model_name, 'info_*.json')):
      model_info = json.load(open(info_file_name))
      #print(model_info)
      dev_correct = model_info['dev_correct']
      dev_total = model_info['dev_total']
      dev_correct_fraction = dev_correct / dev_total
      output.append([dev_correct_fraction, info_file_name])
  output.sort()
  output.reverse()
  return output

get_models_with_dev_correct_fraction()

[[0.8085924439986627,
  'tm_5c89f565babee6f8bbc1f7e4d0c76ca4df515adc/info_3.json'],
 [0.8037445670344366,
  'tm_2769148d0b782749562fe2e53e40214082a280c9/info_3.json'],
 [0.8034102306920762,
  'tm_b7c78733284824761f93f8b8c9ea52af5939749a/info_2.json'],
 [0.8027415580073554,
  'tm_be95b6706b608bd0a6ae5d4d4c4a042a2dda1bf4/info_3.json'],
 [0.8005683717820127,
  'tm_b7c78733284824761f93f8b8c9ea52af5939749a/info_3.json'],
 [0.7990638582413908,
  'tm_264bddc248d1bba4b2bbdd464ea404764edd3393/info_3.json'],
 [0.7982280173854898,
  'tm_264bddc248d1bba4b2bbdd464ea404764edd3393/info_2.json'],
 [0.7980608492143096,
  'tm_7c371d2a549d40cf966421c94d7d05624b6511a6/info_3.json'],
 [0.7978936810431294,
  'tm_b7c78733284824761f93f8b8c9ea52af5939749a/info_1.json'],
 [0.797559344700769,
  'tm_264bddc248d1bba4b2bbdd464ea404764edd3393/info_1.json'],
 [0.7960548311601471,
  'tm_f9eacb599680c28e8be667eb45366b7df8aacef1/info_3.json'],
 [0.7960548311601471,
  'tm_f059315c6c2992e97569934dbb0082b079b67cd6/info_3.j

In [41]:
parameter_info_list = {"train_loss": 39490.94897335768, "train_correct": 95057, "train_total": 107670, "train_confusion": [[54772, 2662, 961, 207], [2729, 25588, 1298, 64], [855, 1441, 12121, 70], [821, 311, 1194, 2576]], "dev_correct": 4837, "dev_total": 5982, "dev_confusion": [[2264, 213, 52, 43], [315, 1777, 99, 14], [73, 173, 548, 19], [65, 19, 60, 248]], "test_correct": 4935, "test_total": 5982, "test_confusion": [[2001, 166, 92, 16], [223, 1434, 134, 11], [125, 126, 1387, 17], [42, 17, 78, 113]], "epoch_start_time": "2019-04-04T17:19:29.213080+00:00", "epoch_start_timestamp": 1554398369, "epoch_end_time": "2019-04-04T18:13:16.462443+00:00", "epoch_end_timestamp": 1554401596}

train_data,dev_data,test_data = get_data_for_parameters(parameter_info_list)

TypeError: string indices must be integers

In [39]:
def evaluate_baseline_model_on_dataset(model, dataset, prefix='dev_'):
  confusion = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
  correct = 0
  total = 0
  for item in dataset:
    category_tensor = item['category']
    feature_tensor = item['feature']
    if feature_tensor.size()[0] == 0:
        continue
    difficulty = tensor_to_difficulty(category_tensor)
    difficulty_idx = get_difficulty_idx(difficulty)
    predicted_tensor = make_prediction(model, feature_tensor.cuda())
    predicted_difficulty = prediction_to_difficulty(predicted_tensor)
    predicted_difficulty_idx = get_difficulty_idx(predicted_difficulty)
    if predicted_difficulty_idx == difficulty_idx:
      correct += 1
    total += 1
    confusion[difficulty_idx][predicted_difficulty_idx] += 1
  return {
    prefix + 'correct': correct,
    prefix + 'total': total,
    prefix + 'confusion': confusion,
  }


NameError: name 'dev_data' is not defined